In [10]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [11]:
spark = SparkSession.builder.appName("twitter_transformation").getOrCreate()

In [12]:
df_tweet = spark.read.json("../../datalake/silver/twitter_datascience/tweet")

In [13]:
df_tweet.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- process_date: date (nullable = true)



In [18]:
tweet_conversas = df_tweet.alias("tweet").groupBy(f.to_date("created_at").alias("created_date")).agg(
    f.countDistinct("author_id").alias("n_tweets"),
    f.sum("like_count").alias("n_like"),
    f.sum("quote_count").alias("n_quote"),
    f.sum("reply_count").alias("n_replay"),
    f.sum("retweet_count").alias("n_retweet")
).withColumn("weekday", f.date_format("created_date", "E"))

In [19]:
tweet_conversas.show()

+------------+--------+------+-------+--------+---------+-------+
|created_date|n_tweets|n_like|n_quote|n_replay|n_retweet|weekday|
+------------+--------+------+-------+--------+---------+-------+
|  2023-09-14|       9|  3744|   4536|    4851|     4104|    Thu|
|  2023-09-19|      27|  6120|   6344|    7584|     5892|    Tue|
|  2023-09-03|      20| 19887|  26334|   20811|    21210|    Sun|
|  2023-08-28|      19| 23218|  22646|   25272|    20462|    Mon|
|  2023-09-17|      17|  5538|   5166|    4632|     6288|    Sun|
|  2023-09-12|      40| 27181|  25520|   25542|    27533|    Tue|
|  2023-09-08|      10|  8025|   7935|   10980|     7635|    Fri|
|  2023-09-01|       9| 12562|  11396|   11880|    11506|    Fri|
|  2023-09-20|      19|  3980|   4268|    4008|     3612|    Wed|
|  2023-09-10|      25| 23049|  20020|   17992|    16497|    Sun|
|  2023-09-18|      18|  4970|   4470|    5550|     6490|    Mon|
|  2023-09-21|      23|  4440|   4512|    4635|     3633|    Thu|
|  2023-09